In [32]:
import os,sys
from datetime import datetime

import numpy as np
from sacred import Experiment
from sacred.commands import print_config
from sacred.observers import FileStorageObserver
from torch.nn.utils import clip_grad_norm_
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

from evaluate import evaluate
from onsets_and_frames import *


In [ ]:
sys.path.append(PACKAGE_PARENT = '..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser(__file__))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

In [43]:
sys.path

['/home/sgu/onsets-and-frames',
 '/home/sgu/onsets-and-frames',
 '/home/sgu/spark-2.4.3-bin-without-hadoop-scala-2.12/python/lib/py4j-0.10.7-src.zip',
 '/home/sgu/spark-2.4.3-bin-without-hadoop-scala-2.12/python',
 '/home/sgu/miniconda3/envs/cs224n/lib/python37.zip',
 '/home/sgu/miniconda3/envs/cs224n/lib/python3.7',
 '/home/sgu/miniconda3/envs/cs224n/lib/python3.7/lib-dynload',
 '',
 '/home/sgu/miniconda3/envs/cs224n/lib/python3.7/site-packages',
 '/home/sgu/miniconda3/envs/cs224n/lib/python3.7/site-packages/IPython/extensions',
 '/home/sgu/.ipython',
 'onsets-and-frame',
 'onsets-and-frame/mel.py',
 '/home/sgu/onsets-and-frame/mel.py',
 '/home/sgu/onsets-and-frames',
 '/home/sgu/onsets-and-frames/mel.py',
 '/home/sgu/onsets-and-frames',
 '/home/sgu/onsets-and-frames/mel',
 '/home/sgu/onsets-and-frames/mel.py',
 '/home/sgu/onsets-and-frames/mel.py',
 '/home/sgu/onsets-and-frames']

In [47]:
sys.path.append("/home/sgu/onsets-and-frames")
from onsets_and_frames.mel  import melspectrogram

In [3]:
    logdir = 'runs/notebook-nn-arch'   
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    iterations = 500000
    resume_iteration = None
    checkpoint_interval = 1000
    train_on = 'MAPS' #sgu

    batch_size = 8
    sequence_length = 327680
    model_complexity = 8 #sgu: this number x 16 is the model_size

    if torch.cuda.is_available() and torch.cuda.get_device_properties(torch.cuda.current_device()).total_memory < 10e9:
        batch_size //= 2
        sequence_length //= 2
        print(f'Reducing batch size to {batch_size} and sequence_length to {sequence_length} to save memory')

    learning_rate = 0.0006
    learning_rate_decay_steps = 10000
    learning_rate_decay_rate = 0.98

    leave_one_out = None

    clip_gradient_norm = 3

    validation_length = sequence_length
    validation_interval = 500

In [4]:
dataset = MAPS(groups=['AkPnBcht', 'AkPnBsdf'], sequence_length=sequence_length) #sgu 
validation_dataset = MAPS(groups=['ENSTDkAm'], sequence_length=validation_length) #sgu
loader = DataLoader(dataset, batch_size, shuffle=True, drop_last=True)

Loading group AkPnBsdf:   0%|          | 0/30 [00:00<?, ?it/s]

Loading 2 groups of MAPS at data/MAPS


Loading group ENSTDkAm: 100%|██████████| 30/30 [00:00<00:00, 217.57it/s]

Loading 1 group of MAPS at data/MAPS


In [25]:
        model = OnsetsAndFrames(N_MELS, MAX_MIDI - MIN_MIDI + 1, model_complexity).to(device)
        optimizer = torch.optim.Adam(model.parameters(), learning_rate)
        resume_iteration = 0
        scheduler = StepLR(optimizer, step_size=learning_rate_decay_steps, gamma=learning_rate_decay_rate)

In [26]:
ds = iter(loader)
d = next(ds)

In [30]:
d['audio'].shape #(batch_size, seqence_length)

torch.Size([8, 327680])

In [49]:
batch = d
audio_label = batch['audio']
nset_label = batch['onset']
offset_label = batch['offset']
frame_label = batch['frame']
velocity_label = batch['velocity']
mel = melspectrogram(audio_label.reshape(-1, audio_label.shape[-1])[:, :-1]).transpose(-1, -2)
onset_pred, offset_pred, _, frame_pred, velocity_pred = model(mel)

In [51]:
from torchviz import make_dot
make_dot(onset_pred)

RecursionError: maximum recursion depth exceeded while calling a Python object